In [1]:
import os   
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np
import imageio
from tqdm import trange
from torch.utils.data import DataLoader, Dataset
import tensorflow.compat.v1 as tf
import pickle
import gc
import pickle
import yaml
from torch import nn
import tensorflow.compat.v1 as tf

import os, sys
os.environ["CUDA_VISIBLE_DEVICES"]='0'

2024-09-10 19:11:48.939584: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-10 19:11:48.985943: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-10 19:11:49.678127: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.logger import Logger, Visualizer
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.sync_batchnorm import DataParallelWithCallback
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.modules.RNN_prediction_module import PredictionModule
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.augmentation import SelectRandomFrames, SelectFirstFrames_two, VideoToTensor
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.frames_dataset import FramesDataset
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.PREDICTOR.Source_Model.VRNN import build_vrnn, get_config
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.PREDICTOR.Source_Model.VRNN_prediction import VRNN_predict
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.PREDICTOR.Source_Model.prediction_toplevel import KPDataset,get_data_from_dataloader_60
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.modules.generator import OcclusionAwareGenerator,calculate_frechet_distance,compute_fvd
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.modules.keypoint_detector import KPDetector
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.logger import Logger, Visualizer, Visualizer_slow


In [ ]:
with open("kp_train_3883_vox.pkl", "rb") as f:
    kp_time_series = pickle.load(f)
len(kp_time_series)

In [7]:
# convert list of keypoints to dictionary: 
for video_idx in range(len(kp_time_series)):
    kp_time_series[video_idx] = kp_time_series[video_idx]['kp']

kp_dict_init = []
for video_idx in range(len(kp_time_series)): # 
    init_mean = []
    init_jacobian = []
    for frame_idx in range(len(kp_time_series[video_idx])):
        kp_mean = kp_time_series[video_idx][frame_idx]['value'].reshape(1,10,2)
        kp_mean = torch.tensor(kp_mean)
        kp_jacobian = kp_time_series[video_idx][frame_idx]['jacobian'].reshape(1,10,2,2)
        kp_jacobian = torch.tensor(kp_jacobian)

        init_mean.append(kp_mean)
        init_jacobian.append(kp_jacobian)

    init_mean = torch.cat(init_mean)
    init_jacobian = torch.cat(init_jacobian)

    init_mean = torch.reshape(init_mean,(1,init_mean.shape[0],init_mean.shape[1],init_mean.shape[2]))
    init_jacobian = torch.reshape(init_jacobian,(1,init_jacobian.shape[0],10,2,2))

    if torch.cuda.is_available():
        # add tensor to cuda
        init_mean = init_mean.to('cuda:0')
        init_jacobian = init_jacobian.to('cuda:0')

    kp_dict_both = {"value":init_mean,"jacobian":init_jacobian}
    kp_dict_init.append(kp_dict_both)

NameError: name 'kp_time_series' is not defined